<a href="https://colab.research.google.com/github/vincent4u/CE807_Text_Analytics/blob/main/check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
import numpy as np
import os
import pickle
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix



import seaborn as sns
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
import lightgbm as lgb
from sklearn.multioutput import MultiOutputRegressor

from scipy.stats import pearsonr
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [ ]:
student_id = 2314555
np.random.seed(student_id)

train_df = pd.read_csv('/content/train.csv')
#train_df.head(20)

val_df=pd.read_csv('/content/valid.csv')
#val_df.head(20)

test_df=pd.read_csv('/content/test.csv')
#test_df.head(20)

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup

def preprocess_text(text):
    # Remove specific tokens
    text = text.replace("SDATA_5 :", "").replace("EDATA_5", "").replace("NEWLINE_TOKEN", "")


    # Remove special characters and punctuation marks using regular expressions
    text = re.sub(r'[^\w\s]', '', text)

    # Remove URLs, email addresses, and phone numbers
    text = re.sub(r"http\S+|www\S+|\S+@\S+|\d{10}", "", text)

    # Remove numbers
    text = re.sub(r"\d+", "", text)

    # Remove non-ASCII characters
    text = text.encode("ascii", "ignore").decode()

    # Convert the text to lowercase
    text = text.lower()

    # Remove extra white spaces
    text = re.sub(r"\s+", " ", text)

    # Tokenize the text
    tokens = nltk.word_tokenize(text)

    # Remove stop words
    stop_words = set(stopwords.words("english"))
    tokens = [word for word in tokens if word not in stop_words]

    # Apply lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # Join the tokens back into a single string
    preprocessed_text = " ".join(tokens)

    return preprocessed_text

# Preprocess the text
train_df['preprocessed_comment'] = train_df['comment'].apply(preprocess_text)
val_df['preprocessed_comment'] = val_df['comment'].apply(preprocess_text)

print(train_df['preprocessed_comment'])

0       kindergarden ignorantsve added pakistani artic...
1                   violate rrv rule may banned wikipedia
2                     judging message tp could use advice
3                                     love fatty sex mmmm
4        please could explain thinking warning page thank
                              ...                        
8694    utci beg differ history hugely important licen...
8695           penis sex grawp haeppenispenisvaginavagina
8696                      thats exactly gamer dying breed
8697                          article unless fact support
8698                milton friedman lp also measure thing
Name: preprocessed_comment, Length: 8699, dtype: object


In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer


#good
# best_params = {
#     "objective": "reg:squarederror",
#     "n_estimators": 649,
#     "verbosity": 0,
#     "learning_rate": 0.029624377829993845,
#     "max_depth": 5,
#     "subsample":  0.9815939695298614,
#     "colsample_bytree": 0.5082728972365318,
#     "min_child_weight": 8
# }
best_params = {
    "objective": "reg:squarederror",
    "n_estimators": 750,
    "verbosity": 0,
    "learning_rate": 0.009624377829993845,
    "max_depth": 9,
    "subsample":  0.9815939695298614,
    "colsample_bytree": 0.5082728972365318,
    "min_child_weight": 8
}



# Create an XGBRegressor model with the best hyperparameters
#vectorizer = TfidfVectorizer()
vectorizer = CountVectorizer()
# Fit the vectorizer on the training data and transform the training data
train_features = vectorizer.fit_transform(train_df['preprocessed_comment'])

# Transform the validation data using the fitted vectorizer
val_features = vectorizer.transform(val_df['preprocessed_comment'])

# Create an instance of XGBClassifier with the best parameters
model = XGBClassifier(**best_params, tree_method='hist')

# Fit the model on the training features and labels
model.fit(train_features, train_df['toxicity'])

# Predict on the validation features
val_pred = model.predict(val_features)


# Calculate metrics
accuracy = accuracy_score(val_df['toxicity'], val_pred)
recall = recall_score(val_df['toxicity'], val_pred, average='macro')
precision = precision_score(val_df['toxicity'], val_pred, average='macro')
f1 = f1_score(val_df['toxicity'], val_pred, average='macro')

# Print metrics
print('Accuracy:', accuracy)
print('Recall (macro):', recall)
print('Precision (macro):', precision)
print('F1 Score (macro):', f1)

# Compute and print confusion matrix
labels = np.unique(np.concatenate((val_df['toxicity'], val_pred)))
cm = confusion_matrix(val_df['toxicity'], val_pred, labels=labels)
print('Confusion Matrix:')
print(cm)



Accuracy: 0.8684931506849315
Recall (macro): 0.49980291683090267
Precision (macro): 0.434395340870161
F1 Score (macro): 0.46480938416422285
Confusion Matrix:
[[2536    1]
 [ 383    0]]


In [ ]:
import numpy as np
from sklearn.feature_selection import RFE
from xgboost import XGBClassifier
from sklearn.feature_extraction.text import CountVectorizer


# Create an instance of CountVectorizer
vectorizer = CountVectorizer()

# Fit the vectorizer on the training data and transform the training data
train_features = vectorizer.fit_transform(train_df['preprocessed_comment'])

# Transform the validation data using the fitted vectorizer
val_features = vectorizer.transform(val_df['preprocessed_comment'])

# Create an instance of XGBClassifier with the best parameters
model = XGBClassifier(tree_method='hist')

# Fit the model on the training features and labels
model.fit(train_features, train_df['toxicity'])

# Predict on the validation features
val_pred = model.predict(val_features)

# Calculate metrics
accuracy = accuracy_score(val_df['toxicity'], val_pred)
recall = recall_score(val_df['toxicity'], val_pred, average='macro')
precision = precision_score(val_df['toxicity'], val_pred, average='macro')
f1 = f1_score(val_df['toxicity'], val_pred, average='macro')

# Print metrics
print('Accuracy:', accuracy)
print('Recall (macro):', recall)
print('Precision (macro):', precision)
print('F1 Score (macro):', f1)

# Compute and print confusion matrix
labels = np.unique(np.concatenate((val_df['toxicity'], val_pred)))
cm = confusion_matrix(val_df['toxicity'], val_pred, labels=labels)
print('Confusion Matrix:')
print(cm)

# Perform feature selection with RFE
num_features_to_keep = 1500  # Adjust this value as per your requirements
rfe = RFE(estimator=model, n_features_to_select=num_features_to_keep)

# Fit RFE on the training features and labels
rfe.fit(train_features, train_df['toxicity'])

# Get the selected features
selected_features = train_features[:, rfe.support_]

# Retrain the model using the selected features
model.fit(selected_features, train_df['toxicity'])

# Predict on the validation features using the model trained with selected features
val_features_selected = val_features[:, rfe.support_]
val_pred_selected = model.predict(val_features_selected)

# Calculate metrics on the selected features
accuracy_selected = accuracy_score(val_df['toxicity'], val_pred_selected)
recall_selected = recall_score(val_df['toxicity'], val_pred_selected, average='macro')
precision_selected = precision_score(val_df['toxicity'], val_pred_selected, average='macro')
f1_selected = f1_score(val_df['toxicity'], val_pred_selected, average='macro')

# Print metrics on the selected features
print('Accuracy (selected features):', accuracy_selected)
print('Recall (macro) (selected features):', recall_selected)
print('Precision (macro) (selected features):', precision_selected)
print('F1 Score (macro) (selected features):', f1_selected)

Accuracy: 0.8664383561643836
Recall (macro): 0.4986204178163185
Precision (macro): 0.4342602128389976
F1 Score (macro): 0.46422018348623845
Confusion Matrix:
[[2530    7]
 [ 383    0]]


KeyboardInterrupt: 